In [32]:
from bw_graph_tools.graph_traversal import Edge as GraphEdge
from bw_graph_tools.graph_traversal import NewNodeEachVisitGraphTraversal
from bw_graph_tools.graph_traversal import Node as GraphNode

In [33]:
import json
from copy import deepcopy
from abc import abstractmethod

def identify_activity_type(activity):
    """Return the activity type based on its naming."""
    name = activity["name"]
    if "treatment of" in name:
        return "treatment"
    elif "market for" in name:
        # if not "to generic" in name:  # these are not markets, but also transferring activities
        return "market"
    elif "market group" in name:
        # if not "to generic" in name:
        return "marketgroup"
    else:
        return "production"
    
class BaseGraph(object):
    def __init__(self):
        self.json_data = None
        # stores previous graphs, if any, and enables back/forward buttons
        self.stack = []
        # stores graphs that can be returned to after having used the "back" button
        self.forward_stack = []

    def update(self, delete_unstacked: bool = True) -> None:
        self.store_previous()
        if delete_unstacked:
            self.forward_stack = []

    def forward(self) -> bool:
        """Go forward, if previously gone back."""
        if not self.forward_stack:
            return False
        self.retrieve_future()
        self.update(delete_unstacked=False)
        return True

    def back(self) -> bool:
        """Go back to previous graph, if any."""
        if len(self.stack) <= 1:
            return False
        self.store_future()
        self.update(delete_unstacked=False)
        return True

    def store_previous(self) -> None:
        """Store the current graph in the"""
        self.stack.append((deepcopy(self.json_data)))

    def store_future(self) -> None:
        """When going back, store current data in a queue."""
        self.forward_stack.append(self.stack.pop())
        self.json_data = self.stack.pop()

    def retrieve_future(self) -> None:
        """Extract the last graph from the queue."""
        self.json_data = self.forward_stack.pop()

    @abstractmethod
    def new_graph(self, *args, **kwargs) -> None:
        pass

    def save_json_to_file(self, filename: str = "graph_data.json") -> None:
        """Writes the current model´s JSON representation to the specifies file."""
        if self.json_data:
            filepath = os.path.join(os.path.dirname(__file__), filename)
            with open(filepath, "w") as outfile:
                json.dump(self.json_data, outfile)
                
class Graph(BaseGraph):
    """
    Python side representation of the graph.
    Functionality for graph navigation (e.g. adding and removing nodes).
    A JSON representation of the graph (edges and nodes) enables its use in javascript/html/css.
    """

    def new_graph(self, data):
        self.json_data = Graph.get_json_data(data)
        self.update()

    @staticmethod
    def get_json_data(data) -> str:
        """Transform graph traversal output to JSON data.

        We use the [dagre](https://github.com/dagrejs/dagre) javascript library for rendering directed graphs. We need to provide the following:

        ```python
        {
            'max_impact': float,  # Total LCA score,
            'title': str,  # Graph title
            'edges': [{
                'source_id': int,  # Unique ID of producer of material or energy in graph
                'target_id': int,  # Unique ID of consumer of material or energy in graph
                'weight': float,  #  In graph units, relative to `max_edge_width`
                'label': str,  # HTML label
                'product': str,  # The label of the flowing material or energy
                'class': str,  # "benefit" or "impact"; controls styling
                'label': str,  # HTML label
                'toottip': str,  # HTML tooltip
            }],
            'nodes': [{
                'direct_emissions_score_normalized': float,  # Fraction of total LCA score from direct emissions
                'product': str,  # Reference product label, if any
                'location': str,  # Location, if any
                'id': int,  # Graph traversal ID
                'database_id': int,  # Node ID in SQLite database
                'database': str,  # Database name
                'class': str,  # Enumerated set of class label strings
                'label': str,  # HTML label including name and location
                'toottip': str,  # HTML tooltip
            }],
        }

        ```

        """
        lca_score = data["metadata"]["lca"].score
        lcia_unit = data["metadata"]["unit"]
        demand = data["metadata"]["lca"].demand

        def convert_edge_to_json(
            edge: GraphEdge,
            nodes: dict[int, GraphNode],
            total_score: float,
            lcia_unit: str,
            max_edge_width: int = 1,
        ) -> dict:
            cum_score = nodes[edge.producer_unique_id].cumulative_score
            unit = bd.get_node(
                id=nodes[edge.producer_unique_id].reference_product_datapackage_id
            ).get("unit", "(unknown)")
            return {
                "source_id": edge.producer_unique_id,
                "target_id": edge.consumer_unique_id,
                "amount": edge.amount,
                "weight": abs(cum_score / total_score) * max_edge_width,
                "label": f"{round(cum_score, 3)} {lcia_unit}",
                "class": "benefit" if cum_score < 0 else "impact",
                "tooltip": f"<b>{round(cum_score, 3)} {lcia_unit}</b> ({edge.amount:.2g} {unit})",
            }

        def convert_node_to_json(
            graph_node: GraphNode,
            total_score: float,
            fu: dict,
            lcia_unit: str,
            max_name_length: int = 20,
        ) -> dict:
            db_node = bd.get_node(id=graph_node.activity_datapackage_id)
            data = {
                "direct_emissions_score_normalized": graph_node.direct_emissions_score
                / (total_score or 1),
                "direct_emissions_score": graph_node.direct_emissions_score,
                "cumulative_score": graph_node.cumulative_score,
                "cumulative_score_normalized": graph_node.cumulative_score
                / (total_score or 1),
                "product": db_node.get("reference product", ""),
                "location": db_node.get("location", "(unknown)"),
                "id": graph_node.unique_id,
                "database_id": graph_node.activity_datapackage_id,
                "database": db_node["database"],
                "class": (
                    "demand"
                    if graph_node.activity_datapackage_id in fu
                    else identify_activity_type(db_node)
                ),
                "name": db_node.get("name", "(unnamed)"),
            }
            frac_dir_score = round(data["direct_emissions_score_normalized"] * 100, 2)
            dir_score = round(data["direct_emissions_score"], 3)
            frac_cum_score = round(data["cumulative_score_normalized"] * 100, 2)
            cum_score = round(data["cumulative_score"], 3)
            data[
                "label"
            ] = f"""{db_node['name'][:max_name_length]}
{data['location']}
{frac_dir_score}%"""
            data[
                "tooltip"
            ] = f"""
                <b>{data['name']}</b>
                <br>Individual impact: {dir_score} {lcia_unit} ({frac_dir_score }%)
                <br>Cumulative impact: {cum_score} {lcia_unit} ({frac_cum_score}%)
            """
            return data

        json_data = {
            "nodes": [
                convert_node_to_json(node, lca_score, demand, lcia_unit)
                for idx, node in data["nodes"].items()
                if idx != -1
            ],
            "edges": [
                convert_edge_to_json(edge, data["nodes"], lca_score, lcia_unit)
                for edge in data["edges"]
                if edge.producer_index != -1 and edge.consumer_index != -1
            ],
            "title": "Sankey graph result",
            # "title": self.build_title(demand, lca_score, lcia_unit),
        }

        return json.dumps(json_data)

    def build_title(self, demand: tuple, lca_score: float, lcia_unit: str) -> str:
        act, amount = demand[0], demand[1]
        if type(act) is tuple or type(act) is int:
            act = bd.get_activity(act)
        format_str = (
            "Reference flow: {:.2g} {} {} | {} | {} <br>" "Total impact: {:.2g} {}"
        )
        return format_str.format(
            amount,
            act.get("unit"),
            act.get("reference product") or act.get("name"),
            act.get("name"),
            act.get("location"),
            lca_score,
            lcia_unit,
        )

In [50]:
import bw2data as bd

bd.projects.set_current("timex")


In [62]:
import bw2calc as bc

act = bd.Database("ecoinvent-3.10-cutoff").random()
lca = bc.LCA(demand={act: 1}, method=bd.methods.random())
lca.lci(factorize=True)
lca.lcia()

/Users/timodiepers/anaconda3/envs/timex/lib/python3.10/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)
/Users/timodiepers/anaconda3/envs/timex/lib/python3.10/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 5.51e+12)
  warnings.warn(msg, UmfpackWarning)


In [76]:
list(act.exchanges())

[Exchange: 1.0 unit 'market for skidder' (unit, GLO, None) to 'market for skidder' (unit, GLO, None)>,
 Exchange: 1.0 unit 'skidder production' (unit, GLO, None) to 'market for skidder' (unit, GLO, None)>,
 Exchange: 217.2 ton kilometer 'market group for transport, freight train' (ton kilometer, GLO, None) to 'market for skidder' (unit, GLO, None)>,
 Exchange: 14.4 ton kilometer 'market group for transport, freight, inland waterways, barge' (ton kilometer, GLO, None) to 'market for skidder' (unit, GLO, None)>,
 Exchange: 117.6 ton kilometer 'market group for transport, freight, light commercial vehicle' (ton kilometer, GLO, None) to 'market for skidder' (unit, GLO, None)>,
 Exchange: 3786.0 ton kilometer 'market group for transport, freight, lorry, unspecified' (ton kilometer, GLO, None) to 'market for skidder' (unit, GLO, None)>,
 Exchange: 8841.6 ton kilometer 'market for transport, freight, sea, container ship' (ton kilometer, GLO, None) to 'market for skidder' (unit, GLO, None)>]

In [64]:
data = NewNodeEachVisitGraphTraversal.calculate(
                    lca_object=lca, max_depth=2,
                )

/var/folders/l1/k90rhb0j0ns58y35ymznsd700000gn/T/ipykernel_94768/3031763972.py:1: DeprecationWarning: Use `NewNodeEachVisitGraphTraversal(lca, settings)` instead of `NNEVGT().calculate(stuff)`
  data = NewNodeEachVisitGraphTraversal.calculate(
/Users/timodiepers/anaconda3/envs/timex/lib/python3.10/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 5.51e+12)
  warnings.warn(msg, UmfpackWarning)


In [65]:
data

{'nodes': {-1: Node(unique_id=-1, activity_datapackage_id=-1, activity_index=-1, reference_product_datapackage_id=-1, reference_product_index=-1, reference_product_production_amount=1.0, depth=0, supply_amount=1.0, cumulative_score=196467.08066629138, direct_emissions_score=0.0, max_depth=None, direct_emissions_score_outside_specific_flows=0.0, remaining_cumulative_score_outside_specific_flows=0.0, terminal=False),
  0: Node(unique_id=0, activity_datapackage_id=14014, activity_index=9651, reference_product_datapackage_id=14014, reference_product_index=9651, reference_product_production_amount=1.0, depth=1, supply_amount=1.0, cumulative_score=196467.08066629135, direct_emissions_score=0.0, max_depth=None, direct_emissions_score_outside_specific_flows=0.0, remaining_cumulative_score_outside_specific_flows=196467.08066629135, terminal=False),
  1: Node(unique_id=1, activity_datapackage_id=12751, activity_index=8388, reference_product_datapackage_id=12751, reference_product_index=8388, ref

In [90]:
list(bd.methods)

[('CML v4.8 2016 no LT',
  'acidification no LT',
  'acidification (incl. fate, average Europe total, A&B) no LT'),
 ('CML v4.8 2016 no LT',
  'climate change no LT',
  'global warming potential (GWP100) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: freshwater no LT',
  'freshwater aquatic ecotoxicity (FAETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: marine no LT',
  'marine aquatic ecotoxicity (MAETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'ecotoxicity: terrestrial no LT',
  'terrestrial ecotoxicity (TETP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'energy resources: non-renewable no LT',
  'abiotic depletion potential (ADP): fossil fuels no LT'),
 ('CML v4.8 2016 no LT',
  'eutrophication no LT',
  'eutrophication (fate not incl.) no LT'),
 ('CML v4.8 2016 no LT',
  'human toxicity no LT',
  'human toxicity (HTP inf) no LT'),
 ('CML v4.8 2016 no LT',
  'material resources: metals/minerals no LT',
  'abiotic depletion potential (ADP): elements (ultimate reserves) no LT')

In [67]:
lca.score

196467.08066629138

In [74]:
data["edges"]

[Edge(consumer_index=-1, consumer_unique_id=-1, producer_index=9651, producer_unique_id=0, product_index=9651, amount=1),
 Edge(consumer_index=9651, consumer_unique_id=0, producer_index=8388, producer_unique_id=1, product_index=8388, amount=3786.0),
 Edge(consumer_index=9651, consumer_unique_id=0, producer_index=16738, producer_unique_id=2, product_index=16738, amount=1.0)]

In [83]:
df_data = {
    "Producer Name": [],
    "Producer Product": [],
    "Producer Location": [],
    "Consumer Name": [],
    "Consumer Product": [],
    "Consumer Location": [],
    "Score": [],
}

for edge in data["edges"]:
    if edge.producer_index != -1 and edge.consumer_index != -1:
        producer_node = bd.get_node(
            id=data["nodes"][edge.producer_unique_id].activity_datapackage_id
        )
        consumer_node = bd.get_node(
            id=data["nodes"][edge.consumer_unique_id].activity_datapackage_id
        )

        df_data["Producer Name"].append(producer_node.get("name", "(unnamed)"))
        df_data["Producer Product"].append(producer_node.get("reference product", ""))
        df_data["Producer Location"].append(producer_node.get("location", "(unknown)"))
        df_data["Consumer Name"].append(consumer_node.get("name", "(unnamed)"))
        df_data["Consumer Product"].append(consumer_node.get("reference product", ""))
        df_data["Consumer Location"].append(consumer_node.get("location", "(unknown)"))
        df_data["Score"].append(data["nodes"][edge.producer_unique_id].cumulative_score)

In [84]:
import pandas as pd
pd.DataFrame(df_data)

,Producer Name,Producer Product,Producer Location,Consumer Name,Consumer Product,Consumer Location,Score
0,"market group for transport, freight, lorry, un...","transport, freight, lorry, unspecified",GLO,market for skidder,skidder,GLO,1495.504672
1,skidder production,skidder,GLO,market for skidder,skidder,GLO,194102.495712


In [60]:
cum_score = nodes[edge.producer_unique_id].cumulative_score

data["edges"][1]['cumulative_score']

TypeError: 'Edge' object is not subscriptable

In [ ]:
df_data = {"Producer": [], "Consumer": [], "Impact": []}
for edge in data.edges:
    


In [40]:
# from bw_graph_tools.graph_traversal.settings import GraphTraversalSettings

# gt = NewNodeEachVisitGraphTraversal(lca, GraphTraversalSettings())
# gt.traverse()
# gt.nodes

In [41]:
# store the metadata from this calculation
data["metadata"] = {
    "lca": lca,
    "unit": "kg CO2-eq",
}

In [42]:
g = Graph()
g.new_graph(data)

In [43]:
res = json.loads(g.json_data)

In [48]:
res["edges"]

[{'source_id': 1,
  'target_id': 0,
  'amount': 0.08939731866121292,
  'weight': 0.8683350423488327,
  'label': '0.001 kg CO2-eq',
  'class': 'impact',
  'tooltip': '<b>0.001 kg CO2-eq</b> (0.089 kilogram)'},
 {'source_id': 2,
  'target_id': 0,
  'amount': 0.12772777676582336,
  'weight': 2.514010021360707,
  'label': '0.002 kg CO2-eq',
  'class': 'impact',
  'tooltip': '<b>0.002 kg CO2-eq</b> (0.13 kilogram)'},
 {'source_id': 3,
  'target_id': 0,
  'amount': 0.3990631401538849,
  'weight': 3.4999795970678402,
  'label': '0.003 kg CO2-eq',
  'class': 'impact',
  'tooltip': '<b>0.003 kg CO2-eq</b> (0.4 kilowatt hour)'},
 {'source_id': 4,
  'target_id': 0,
  'amount': 0.3747216761112213,
  'weight': 12.39091251230315,
  'label': '0.01 kg CO2-eq',
  'class': 'impact',
  'tooltip': '<b>0.01 kg CO2-eq</b> (0.37 kilogram)'},
 {'source_id': 5,
  'target_id': 0,
  'amount': 4.000000053405728e-10,
  'weight': 1.164874568938576,
  'label': '0.001 kg CO2-eq',
  'class': 'impact',
  'tooltip': '<b

In [44]:
id_to_database_id = {}
for producer_node in res['nodes']:
    id_to_database_id[producer_node['id']] = producer_node['database_id']

In [122]:
df_data = {"source": [], "target": [], "amount": []}
for edge in res["edges"]:
    df_data["source"].append(id_to_database_id[edge["source_id"]])
    df_data["target"].append(id_to_database_id[edge["target_id"]])
    df_data["amount"].append(edge["amount"])

In [128]:
import pandas as pd
df = pd.DataFrame(df_data)
df.sort_values(by="amount", ascending=False)

,source,target,amount
1,23728,20930,9.560936e-01
0,12450,20930,1.867658e-08


In [102]:
res["edges"]

[{'source_id': 1,
  'target_id': 0,
  'amount': 1.8676577612333276e-08,
  'weight': 0.8299088428912493,
  'label': '0.0 kg CO2-eq',
  'class': 'impact',
  'tooltip': '<b>0.0 kg CO2-eq</b> (1.9e-08 kilometer)'},
 {'source_id': 2,
  'target_id': 0,
  'amount': 0.9560936021853338,
  'weight': 38.980582169785585,
  'label': '0.004 kg CO2-eq',
  'class': 'impact',
  'tooltip': '<b>0.004 kg CO2-eq</b> (0.96 kilowatt hour)'}]